In [1]:
!curl -s 'https://godbolt.org/api/compilers/c' | grep -i "x86-64 clang"

cclang_assertions_trunk  | x86-64 clang (assertions trunk)
cclang_dang              | x86-64 clang (thephd.dev)
cclang_trunk             | x86-64 clang (trunk)
cclang_widberg           | x86-64 clang (widberg)
cclang1000               | x86-64 clang 10.0.0
cclang1001               | x86-64 clang 10.0.1
cclang1100               | x86-64 clang 11.0.0
cclang1101               | x86-64 clang 11.0.1
cclang1200               | x86-64 clang 12.0.0
cclang1201               | x86-64 clang 12.0.1
cclang1300               | x86-64 clang 13.0.0
cclang1301               | x86-64 clang 13.0.1
cclang1400               | x86-64 clang 14.0.0
cclang1500               | x86-64 clang 15.0.0
cclang1600               | x86-64 clang 16.0.0
cclang1701               | x86-64 clang 17.0.1
cclang1810               | x86-64 clang 18.1.0
cclang1910               | x86-64 clang 19.1.0
cclang2010               | x86-64 clang 20.1.0
cclang30                 | x86-64 clang 3.0.0
cclang31                 | x86-64 clang

In [3]:
import requests
import json
from tabulate import tabulate
import csv
import sys
import re

In [4]:
TEST_CODE = "int main() { return 0; }"

# Flags to test
FLAGS = [
    "-fsanitize=address",
    "-fno-omit-frame-pointer",
    "-fsanitize-recover=address",
    "-Wall -Wextra",
    # UBSan flags
    "-fsanitize=undefined",
    "-fno-sanitize-recover=all",
    "-fsanitize=float-divide-by-zero",
    "-fsanitize=float-cast-overflow",
    "-fno-sanitize=null",
    "-fno-sanitize=alignment"
]

# use-after-scope needs to be combined in order to work
COMBINED_FLAGS = {
    "-fsanitize-address-use-after-scope": "-fsanitize=address -fsanitize-address-use-after-scope",
    "-fsanitize=float-divide-by-zero": "-fsanitize=undefined -fsanitize=float-divide-by-zero",
    "-fsanitize=float-cast-overflow": "-fsanitize=undefined -fsanitize=float-cast-overflow"
}

# Compiler ID to name mapping
COMPILER_MAPPING = {
    "cclang1000": "clang 10.0.0",
    "cclang1001": "clang 10.0.1",
    "cclang1100": "clang 11.0.0",
    "cclang1101": "clang 11.0.1",
    "cclang1200": "clang 12.0.0",
    "cclang1201": "clang 12.0.1",
    "cclang1300": "clang 13.0.0",
    "cclang1301": "clang 13.0.1",
    "cclang1400": "clang 14.0.0",
    "cclang1500": "clang 15.0.0",
    "cclang1600": "clang 16.0.0",
    "cclang1701": "clang 17.0.1",
    "cclang1810": "clang 18.1.0",
    "cclang1910": "clang 19.1.0",
    "cclang2010": "clang 20.1.0",
    "cclang30": "clang 3.0.0",
    "cclang31": "clang 3.1",
    "cclang32": "clang 3.2",
    "cclang33": "clang 3.3",
    "cclang341": "clang 3.4.1",
    "cclang350": "clang 3.5",
    "cclang351": "clang 3.5.1",
    "cclang352": "clang 3.5.2",
    "cclang36x": "clang 3.6",
    "cclang37x": "clang 3.7",
    "cclang371": "clang 3.7.1",
    "cclang380": "clang 3.8",
    "cclang381": "clang 3.8.1",
    "cclang390": "clang 3.9.0",
    "cclang391": "clang 3.9.1",
    "cclang400": "clang 4.0.0",
    "cclang401": "clang 4.0.1",
    "cclang500": "clang 5.0.0",
    "cclang501": "clang 5.0.1",
    "cclang502": "clang 5.0.2",
    "cclang600": "clang 6.0.0",
    "cclang601": "clang 6.0.1",
    "cclang700": "clang 7.0.0",
    "cclang701": "clang 7.0.1",
    "cclang710": "clang 7.1.0",
    "cclang800": "clang 8.0.0",
    "cclang801": "clang 8.0.1",
    "cclang900": "clang 9.0.0",
    "cclang901": "clang 9.0.1"
}

In [5]:
def test_flag(compiler_id, flag):
    url = f"https://godbolt.org/api/compiler/{compiler_id}/compile"

    payload = {
        "source": TEST_CODE,
        "options": {
            "userArguments": flag
        }
    }

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        result = response.json()
        if "stderr" in result and result["stderr"]:
            error_text = result["stderr"][0]["text"] if result["stderr"] else "CHECK ERROR MANUALLY"
            return False, error_text

        return True, ""
    except Exception as e:
        return False, f"API error: {str(e)}"

# used for sorting
def parse_version(compiler_name):
    version_match = re.search(r'(\d+)(?:\.(\d+))?(?:\.(\d+))?', compiler_name)
    if not version_match:
        return (0, 0, 0)

    major = int(version_match.group(1)) if version_match.group(1) else 0
    if 'x' in compiler_name and version_match.group(1):
        return (major, float('inf'), 0)

    minor = int(version_match.group(2)) if version_match.group(2) else 0
    patch = int(version_match.group(3)) if version_match.group(3) else 0

    return (major, minor, patch)

sorted_compilers = sorted(
    COMPILER_MAPPING.items(),
    key=lambda x: parse_version(x[1])
)

In [6]:
all_results = []
csv_results = []

csv_header = ["Compiler ID", "Compiler Name"]
for flag in FLAGS:
    csv_header.extend([f"{flag} Support", f"{flag} Error"])

for flag in COMBINED_FLAGS:
    csv_header.extend([f"{flag} Support", f"{flag} Error"])

all_flags = " ".join(FLAGS + list(COMBINED_FLAGS.keys()))
csv_header.extend([f"All Flags Support", f"All Flags Error"])

csv_results.append(csv_header)

print(f"TOTAL clang versions {len(COMPILER_MAPPING)}...")
for compiler_id, compiler_name in sorted_compilers:
    print(f"Testing {compiler_id} ({compiler_name})...")
    row_result = [compiler_id, compiler_name]
    table_row = [f"{compiler_id}\n{compiler_name}"]

    # Test each flag individually
    flag_results = {}
    for flag in FLAGS:
        print(f"  Testing flag: {flag}...")
        supported, error = test_flag(compiler_id, flag)

        status = 1 if supported else 0
        table_row.append(status)
        row_result.append(status)
        row_result.append(error if not supported else "")

        flag_results[flag] = (supported, error)

    # Test combined flags
    for flag, combined_flag in COMBINED_FLAGS.items():
        print(f"  Testing combined flag: {flag}...")
        supported, error = test_flag(compiler_id, combined_flag)

        status = 1 if supported else 0
        table_row.append(status)

        # Add to CSV results
        row_result.append(status)
        row_result.append(error if not supported else "")

        flag_results[flag] = (supported, error)

    # Test all flags together - use the main flags and add combined flags only if supported
    all_flags_to_test = FLAGS.copy()
    for flag in COMBINED_FLAGS:
        if flag_results[flag][0]:  # If flag is supported
            all_flags_to_test.append(flag)

    combined_all_flags = " ".join(all_flags_to_test)
    print(f"  Testing all supported flags together...")
    supported, error = test_flag(compiler_id, combined_all_flags)

    status = 1 if supported else 0
    table_row.append(status)

    # Add to CSV results
    row_result.append(status)
    row_result.append(error if not supported else "")

    all_results.append(table_row)
    csv_results.append(row_result)

    # Print progress
    sys.stdout.write('.')
    sys.stdout.flush()


TOTAL clang versions 44...
Testing cclang30 (clang 3.0.0)...
  Testing flag: -fsanitize=address...
  Testing flag: -fno-omit-frame-pointer...
  Testing flag: -fsanitize-recover=address...
  Testing flag: -Wall -Wextra...
  Testing flag: -fsanitize=undefined...
  Testing flag: -fno-sanitize-recover=all...
  Testing flag: -fsanitize=float-divide-by-zero...
  Testing flag: -fsanitize=float-cast-overflow...
  Testing flag: -fno-sanitize=null...
  Testing flag: -fno-sanitize=alignment...
  Testing combined flag: -fsanitize-address-use-after-scope...
  Testing combined flag: -fsanitize=float-divide-by-zero...
  Testing combined flag: -fsanitize=float-cast-overflow...
  Testing all supported flags together...
.Testing cclang31 (clang 3.1)...
  Testing flag: -fsanitize=address...
  Testing flag: -fno-omit-frame-pointer...
  Testing flag: -fsanitize-recover=address...
  Testing flag: -Wall -Wextra...
  Testing flag: -fsanitize=undefined...
  Testing flag: -fno-sanitize-recover=all...
  Testing 

In [7]:
headers = ["Compiler"] + FLAGS + list(COMBINED_FLAGS.keys()) + ["All Flags"]
print("\n\nASan Flag Support Summary:")
print(tabulate(all_results, headers=headers, tablefmt="grid"))



ASan Flag Support Summary:
+--------------+----------------------+---------------------------+------------------------------+-----------------+------------------------+-----------------------------+-----------------------------------+----------------------------------+----------------------+---------------------------+--------------------------------------+-----------------------------------+----------------------------------+-------------+
| Compiler     |   -fsanitize=address |   -fno-omit-frame-pointer |   -fsanitize-recover=address |   -Wall -Wextra |   -fsanitize=undefined |   -fno-sanitize-recover=all |   -fsanitize=float-divide-by-zero |   -fsanitize=float-cast-overflow |   -fno-sanitize=null |   -fno-sanitize=alignment |   -fsanitize-address-use-after-scope |   -fsanitize=float-divide-by-zero |   -fsanitize=float-cast-overflow |   All Flags |
+==============+======================+===========================+==============================+=================+===================

In [8]:
with open('asan_flag_compatibility_clang.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in csv_results:
        csv_writer.writerow(row)



In [9]:
!curl -s 'https://godbolt.org/api/compilers/c' | grep -i "x86-64 gcc"

cgstatic-analysis        | x86-64 gcc (static analysis)
cgsnapshot               | x86-64 gcc (trunk)
cg101                    | x86-64 gcc 10.1
cg102                    | x86-64 gcc 10.2
cg103                    | x86-64 gcc 10.3
cg103assert              | x86-64 gcc 10.3 (assertions)
cg104                    | x86-64 gcc 10.4
cg104assert              | x86-64 gcc 10.4 (assertions)
cg105                    | x86-64 gcc 10.5
cg105assert              | x86-64 gcc 10.5 (assertions)
cg111                    | x86-64 gcc 11.1
cg111assert              | x86-64 gcc 11.1 (assertions)
cg112                    | x86-64 gcc 11.2
cg112assert              | x86-64 gcc 11.2 (assertions)
cg113                    | x86-64 gcc 11.3
cg113assert              | x86-64 gcc 11.3 (assertions)
cg114                    | x86-64 gcc 11.4
cg114assert              | x86-64 gcc 11.4 (assertions)
cg121                    | x86-64 gcc 12.1
cg121assert              | x86-64 gcc 12.1 (assertions)
cg122               

In [10]:
COMPILER_MAPPING = {
   "cg101": "gcc 10.1",
   "cg102": "gcc 10.2",
   "cg103": "gcc 10.3",
   "cg104": "gcc 10.4",
   "cg105": "gcc 10.5",
   "cg111": "gcc 11.1",
   "cg112": "gcc 11.2",
   "cg113": "gcc 11.3",
   "cg114": "gcc 11.4",
   "cg121": "gcc 12.1",
   "cg122": "gcc 12.2",
   "cg123": "gcc 12.3",
   "cg124": "gcc 12.4",
   "cg131": "gcc 13.1",
   "cg132": "gcc 13.2",
   "cg133": "gcc 13.3",
   "cg141": "gcc 14.1",
   "cg142": "gcc 14.2",
   "cg346": "gcc 3.4.6",
   "cg404": "gcc 4.0.4",
   "cg412": "gcc 4.1.2",
   "cg447": "gcc 4.4.7",
   "cg453": "gcc 4.5.3",
   "cg464": "gcc 4.6.4",
   "cg471": "gcc 4.7.1",
   "cg472": "gcc 4.7.2",
   "cg473": "gcc 4.7.3",
   "cg474": "gcc 4.7.4",
   "cg481": "gcc 4.8.1",
   "cg482": "gcc 4.8.2",
   "cg483": "gcc 4.8.3",
   "cg484": "gcc 4.8.4",
   "cg485": "gcc 4.8.5",
   "cg490": "gcc 4.9.0",
   "cg491": "gcc 4.9.1",
   "cg492": "gcc 4.9.2",
   "cg493": "gcc 4.9.3",
   "cg494": "gcc 4.9.4",
   "cg510": "gcc 5.1",
   "cg520": "gcc 5.2",
   "cg530": "gcc 5.3",
   "cg540": "gcc 5.4",
   "cg6": "gcc 6.1",
   "cg62": "gcc 6.2",
   "cg63": "gcc 6.3",
   "cg65": "gcc 6.5",
   "cg71": "gcc 7.1",
   "cg72": "gcc 7.2",
   "cg73": "gcc 7.3",
   "cg74": "gcc 7.4",
   "cg75": "gcc 7.5",
   "cg81": "gcc 8.1",
   "cg82": "gcc 8.2",
   "cg83": "gcc 8.3",
   "cg84": "gcc 8.4",
   "cg85": "gcc 8.5",
   "cg91": "gcc 9.1",
   "cg92": "gcc 9.2",
   "cg93": "gcc 9.3",
   "cg94": "gcc 9.4",
   "cg95": "gcc 9.5"
}
sorted_compilers = sorted(
    COMPILER_MAPPING.items(),
    key=lambda x: parse_version(x[1])
)

In [11]:
all_results = []
csv_results = []

csv_header = ["Compiler ID", "Compiler Name"]
for flag in FLAGS:
    csv_header.extend([f"{flag} Support", f"{flag} Error"])

for flag in COMBINED_FLAGS:
    csv_header.extend([f"{flag} Support", f"{flag} Error"])

all_flags = " ".join(FLAGS + list(COMBINED_FLAGS.keys()))
csv_header.extend([f"All Flags Support", f"All Flags Error"])

csv_results.append(csv_header)

print(f"Testing {len(COMPILER_MAPPING)} GCC versions...")

for compiler_id, compiler_name in sorted_compilers:
    print(f"Testing {compiler_id} ({compiler_name})...")
    row_result = [compiler_id, compiler_name]
    table_row = [f"{compiler_id}\n{compiler_name}"]

    flag_results = {}
    for flag in FLAGS:
        print(f"  Testing flag: {flag}...")
        supported, error = test_flag(compiler_id, flag)

        status = 1 if supported else 0
        table_row.append(status)

        row_result.append(status)
        row_result.append(error if not supported else "")

        flag_results[flag] = (supported, error)

    for flag, combined_flag in COMBINED_FLAGS.items():
        print(f"  Testing combined flag: {flag}...")
        supported, error = test_flag(compiler_id, combined_flag)

        status = 1 if supported else 0
        table_row.append(status)

        row_result.append(status)
        row_result.append(error if not supported else "")

        flag_results[flag] = (supported, error)

    all_flags_to_test = FLAGS.copy()
    for flag in COMBINED_FLAGS:
        if flag_results[flag][0]:
            all_flags_to_test.append(flag)

    combined_all_flags = " ".join(all_flags_to_test)
    print(f"  Testing all supported flags together...")
    supported, error = test_flag(compiler_id, combined_all_flags)

    status = 1 if supported else 0
    table_row.append(status)

    row_result.append(status)
    row_result.append(error if not supported else "")

    all_results.append(table_row)
    csv_results.append(row_result)

    sys.stdout.write('.')
    sys.stdout.flush()


Testing 61 GCC versions...
Testing cg346 (gcc 3.4.6)...
  Testing flag: -fsanitize=address...
  Testing flag: -fno-omit-frame-pointer...
  Testing flag: -fsanitize-recover=address...
  Testing flag: -Wall -Wextra...
  Testing flag: -fsanitize=undefined...
  Testing flag: -fno-sanitize-recover=all...
  Testing flag: -fsanitize=float-divide-by-zero...
  Testing flag: -fsanitize=float-cast-overflow...
  Testing flag: -fno-sanitize=null...
  Testing flag: -fno-sanitize=alignment...
  Testing combined flag: -fsanitize-address-use-after-scope...
  Testing combined flag: -fsanitize=float-divide-by-zero...
  Testing combined flag: -fsanitize=float-cast-overflow...
  Testing all supported flags together...
.Testing cg404 (gcc 4.0.4)...
  Testing flag: -fsanitize=address...
  Testing flag: -fno-omit-frame-pointer...
  Testing flag: -fsanitize-recover=address...
  Testing flag: -Wall -Wextra...
  Testing flag: -fsanitize=undefined...
  Testing flag: -fno-sanitize-recover=all...
  Testing flag: -f

In [12]:
headers = ["Compiler"] + FLAGS + list(COMBINED_FLAGS.keys()) + ["All Flags"]
print("\n\nASan Flag Support Summary:")
print(tabulate(all_results, headers=headers, tablefmt="grid"))



ASan Flag Support Summary:
+------------+----------------------+---------------------------+------------------------------+-----------------+------------------------+-----------------------------+-----------------------------------+----------------------------------+----------------------+---------------------------+--------------------------------------+-----------------------------------+----------------------------------+-------------+
| Compiler   |   -fsanitize=address |   -fno-omit-frame-pointer |   -fsanitize-recover=address |   -Wall -Wextra |   -fsanitize=undefined |   -fno-sanitize-recover=all |   -fsanitize=float-divide-by-zero |   -fsanitize=float-cast-overflow |   -fno-sanitize=null |   -fno-sanitize=alignment |   -fsanitize-address-use-after-scope |   -fsanitize=float-divide-by-zero |   -fsanitize=float-cast-overflow |   All Flags |
+============+======================+===========================+==============================+=================+========================+

In [13]:
with open('asan_flag_compatibility_gcc.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in csv_results:
        csv_writer.writerow(row)

print(f"\nDetailed results saved to asan_flag_compatibility_gcc.csv")


Detailed results saved to asan_flag_compatibility_gcc.csv
